<div>
    <h1>Large Language Models Projects</a></h1>
    <h3>Apply and Implement Strategies for Large Language Models</h3>
    <h2>6.4-Setting UP a NL2SQL project with Ollama. </h2>
    <h3></h3>
    <p>by <b>Pere Martra</b></p>
</div>

Ollama is one of simplest ans easiest to configure model Server that you can use in your development environmet.

In this notebook you are going to use Models from Ollama and create a custom one abel to generate SQL.

**This noteebooks needs to run in a system with ollama installed, not in google Colab, son install ollama in your computer and the you will be able to execute the notebook.** 

![My Image](https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_6-18.jpg?raw=true)

You only need to download Ollama (https://ollama.com/) and follow the instructions in just few minutes your Ollama server will bve ready.

With Ollama installed you just need to install de Python API and load it in the notebook.

Once installed you can Pull the Modeles you are interested in using the command:
**ollama pull < model_name >.**

pere@Peres-MBP ~ % ollama pull llama3

*pulling manifest
pulling ef311de6af9d... 100% ▕████████████████████▏ 5.0 GB                         
pulling 097a36493f71... 100% ▕████████████████████▏ 8.4 KB                         
pulling 109037bec39c... 100% ▕████████████████████▏  136 B                         
pulling 65bb16cf5983... 100% ▕████████████████████▏  109 B                         
pulling 0c2a5137eb3c... 100% ▕████████████████████▏  483 B                         
verifying sha256 digest
writing manifest
removing any unused layers
success*


In [2]:
pip install -q ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00


In [1]:
import ollama

Now is necessary to setup a user_message and instruction for the prompt.

In [2]:
# Define the user message to send.
user_message = "What is the name of the best paid employee?"

model_instructions = """
Your task is to convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word to appropriately answer the question.
- **Return Only SQL Code.
- **Don't add any explanation.
   ### Input
   This SQL query generatd will run on a database whose schema is represented below:

   create table employees(
       ID_Usr INT primary key,-- Unique Id for employee
       name VARCHAR -- Name of employee
       );

   create table salary(
       ID_Usr INT,-- Unique Id for employee
       year DATE, -- Date
       salary FLOAT, --Salary of employee
       foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
       );

   create table studies(
       ID_study INT, -- Unique ID study
       ID_Usr INT, -- ID employee
       educational_level INT,  -- 5=phd, 4=Master, 3=Bachelor
       Institution VARCHAR, --Name of instituon where eployee studied
       Years DATE, -- Date acomplishement stdy
       Speciality VARCHAR, -- Speciality of studies
       primary key (ID_study, ID_Usr), --Primary Key ID_Usr + ID_Study
       foreign key(ID_Usr) references employees (ID_Usr)
       );
"""

In [6]:
response = ollama.generate(model='myllamasql', 
                           system=model_instructions, 
                           prompt = user_message) 
print(response['response'])

SELECT e.name 
FROM employees e 
JOIN salary s ON e.ID_Usr = s.ID_Usr 
ORDER BY s.salary DESC LIMIT 1;


As you noticed you are passing the user question and the model instructions in two different variables. This is due to the template that ollama use to call the model. 

{{ if .System }}<|start_header_id|>system<|end_header_id|>

{{ .System }}<|eot_id|>{{ end }}{{ if .Prompt }}<|start_header_id|>user<|end_header_id|>

{{ .Prompt }}<|eot_id|>{{ end }}<|start_header_id|>assistant<|end_header_id|>



You can create a new model based in one existent and override the variables, to change how the model works. 

In [14]:
#Configuration file based in llama3 and using One Shot Sample. 
modelfile = """
FROM llama3

MESSAGE user How Many employes we have with a salary bigger than 50000?
MESSAGE assistant SELECT COUNT(*) AS total_employees FROM employees e INNER JOIN salary s ON e.ID_Usr = s.ID_Usr WHERE s.salary > 50000;

PARAMETER repeat_penalty 1.2
PARAMETER temperature 0.1
"""

In [15]:
#Creating a new model that have diferent hyperparameters and One shot Sample 
ollama.create(model="llamasql", modelfile=modelfile)

{'status': 'success'}

In [16]:
response = ollama.generate(model='llamasql',
                           system=model_instructions,
                           prompt = user_message)

In [17]:
print(response['response'])

SELECT e.name 
FROM employees e 
JOIN salary s ON e.ID_Usr = s.ID_Usr 
ORDER BY s.salary DESC LIMIT 1;


Is posible to create a new model with a file  stored in a directory. 

Sample of file: 
_____________
FROM llama3

SYSTEM """ Your task is to convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word to appropriately answer the question.
- **Return Only SQL Code. 
- **Don't add any explanation.
   
   This SQL query generatd will run on a database whose schema is represented below:

   create table employees(
       ID_Usr INT primary key,-- Unique Id for employee
       name VARCHAR -- Name of employee
       );

   create table salary(
       ID_Usr INT,-- Unique Id for employee
       year DATE, -- Date
       salary FLOAT, --Salary of employee
       foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
       );

   create table studies(
       ID_study INT, -- Unique ID study
       ID_Usr INT, -- ID employee
       educational_level INT,  -- 5=phd, 4=Master, 3=Bachelor
       Institution VARCHAR, --Name of instituon where eployee studied
       Years DATE, -- Date acomplishement stdy
       Speciality VARCHAR, -- Speciality of studies
       primary key (ID_study, ID_Usr), --Primary Key ID_Usr + ID_Study
       foreign key(ID_Usr) references employees (ID_Usr)
       );"""

MESSAGE user How Many employes we have with a salary bigger than 50000?

MESSAGE assistant """
SELECT COUNT(*) AS total_employees
FROM employees e
INNER JOIN salary s ON e.ID_Usr = s.ID_Usr
WHERE s.salary > 50000;"""

MESSAGE user Return the names of the three people who have had the highest salary increase in the last three years.

MESSAGE assistant """
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.year >= DATE_SUB(CURDATE(), INTERVAL 3 YEAR)
GROUP BY e.name
ORDER BY (MAX(s.salary) - MIN(s.salary)) DESC
LIMIT 3;"""

PARAMETER repeat_penalty 1.2
PARAMETER temperature 0.1
_____________________________

This file contains the system instructions, a pair of shots and two hyperparameters. To create the Model you can use the order: 

**ollama create llamanlsql -f < filename >**
from your terminal. 


In [21]:
#Use this order to call the recent create model that contains the system message. 
#Notice that you don't need to pass the system, just the user message. 
response = ollama.generate(model='llamanlsql',
                           prompt = user_message)

In [ ]:
print(response['response'])

SELECT e.name 
FROM employees e 
JOIN salary s ON e.ID_Usr = s.ID_Usr 
ORDER BY s.salary DESC LIMIT 1;
